# Understanding the Clyfar Fuzzy Logic Inference System (FIS)

*Prepared by Michael Davies, 13/09/2024*

## Introduction

This document provides a walkthrough of the **Clyfar Fuzzy Logic Inference System (FIS)** developed by *John R. Lawson*. The goal is to help you understand the code, its functionality, and the scientific concepts behind it.

**Clyfar** (*Welsh for clever*) is a **Numerical Weather Prediction (NWP)** and **Artificial Intelligence (AI)** hybrid system. It utilizes a set of rules to process weather forecast data and predict maximum ozone levels over the next 14 days. Version 0.9 of Clyfar is based on a **fuzzy-logic inference system (FIS)**, offering an explainable AI model that can be evolved and optimized.

In this prototype, we focus on predicting winter ozone concentrations in the **Uinta Basin, Utah**. Instead of using probabilities, we use the concepts of **possibility and necessity** to communicate uncertainty, which is particularly useful when data is sparse.


## 1. Importing Libraries

---

First, we need to import all the necessary Python libraries required for the **Clyfar** system.


In [1]:
import calendar
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patheffects as pe
import cartopy.geodesic as cgeo
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from PIL import Image
import shapely
import cartopy.feature as cfeature
from synoptic.services import stations_latest, stations_metadata

# Set up matplotlib to use LaTeX fonts
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=12)
plt.rc('mathtext', fontset='cm')
plt.rcParams['figure.dpi'] = 300

# Define the year for data analysis
y = 2022


/opt/miniconda3/envs/clyfar/lib/python3.11/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


ModuleNotFoundError: No module named 'synoptic'

## 2. Defining Helper Functions

---

Several helper functions are defined to support the main functionality of the **Clyfar** system.

### 2.1 `range_from_gaussian_params`

This function calculates approximate ranges for each ozone category based on the centers and sigma values of Gaussian curves.


In [ ]:
def range_from_gaussian_params(centres_dict, sigma_dict) -> dict:
    """
    From dicts of Gaussian-curve centres and sigma values, return approximate ranges.

    Args:
        centres_dict: dict with category names as keys; centres of the Gaussian curves as values
        sigma_dict: dict with category names as keys; sigma values of the Gaussian curves as values

    Returns:
        range_dict: dict with category names as keys; ranges as [lower, upper]
    """
    range_dict = {}
    for cat, centre in centres_dict.items():
        sigma = sigma_dict[cat]
        # Calculate the lower and upper bounds
        lower_raw = centre - 2.0 * sigma
        upper_raw = centre + 2.0 * sigma
        # Round to the nearest multiple of 5
        lower = 5 * round(lower_raw / 5)
        upper = 5 * round(upper_raw / 5)
        range_dict[cat] = [lower, upper]
    return range_dict

### 2.2 `generate_str_from_range`

Generates a string representation of the ranges for each category, suitable for **LaTeX** rendering.


In [ ]:
def generate_str_from_range(range_dict):
    """
    Generate a LaTeX-friendly string from a dictionary of ranges.

    Args:
        range_dict: dict with category names as keys; ranges as [lower, upper]

    Returns:
        range_str_dict: dict with category names as keys; string representations of ranges
    """
    range_str_dict = {}
    for cat, (lower, upper) in range_dict.items():
        approx_str = r'$\approx$'
        range_str_dict[cat] = f"{approx_str}{lower:.0f}--{upper:.0f} ppb"
    return range_str_dict

### 2.3 `maybe_amend_year`

Adjusts the year if the month is December, assuming the data spans over New Year's.

In [ ]:
def maybe_amend_year(month, year):
    """
    Adjust the year if the month is December, assuming the data spans over New Year's.

    Args:
        month: int representing the month
        year: int representing the year

    Returns:
        year: int representing the adjusted year
    """
    if month == 12:
        year += 1
    return year

### 2.4 `compute_necessity_distr`

Computes the necessity distribution from a possibility distribution.


In [ ]:
def select_function(func):
    if func == "max":
        f = np.max
    elif func == "mean":
        f = np.mean
    else:
        raise ValueError("func must be 'max' or 'mean'")
    return f

def compute_necessity_distr(possibility_array, func="max"):
    """
    Compute necessity distribution from a possibility distribution.

    Args:
        possibility_array: Numpy array of possibility values
        func: Function to apply ('max' or 'mean')

    Returns:
        necessity_array: Numpy array of necessity values
    """
    f = select_function(func)
    necessity_array = []
    n_categories = len(possibility_array)

    for i in range(n_categories):
        other_possibilities = [possibility_array[j] for j in range(n_categories) if j != i]
        necessity_array.append(1 - f(other_possibilities))
    return np.array(necessity_array)


### 2.5 `do_normalization`

Normalizes the possibility array to ensure the supremum is 1.


In [ ]:
def do_normalization(possibility_array):
    """
    Rescale the possibility array to ensure sup(possibility) = 1.

    Args:
        possibility_array: Numpy array of possibility values

    Returns:
        possibility_array_norm: Normalized array
    """
    possibility_array_norm = possibility_array / np.max(possibility_array)
    return possibility_array_norm


### 2.6 `create_possibility_array`

Creates an array of possibility values for a given ozone simulation.


In [ ]:
def create_possibility_array(ozone_simulation, ozone, normalize=False):
    """
    Create an array of possibility values.

    Args:
        ozone_simulation: The ozone simulation object
        ozone: The ozone variable
        normalize: Boolean indicating whether to normalize the array

    Returns:
        possibility_array: Numpy array of possibility values
    """
    possibility_array = np.array([k.membership_value[ozone_simulation] for k in ozone.terms.values()])
    if normalize:
        possibility_array = do_normalization(possibility_array)
    return possibility_array


### 2.7 `compute_possibility_necessity_df`

Computes a DataFrame containing possibility and necessity values for each ozone category.

In [ ]:
def compute_possibility_necessity_df(ozone_simulation, ozone, normalize=False):
    """
    Compute possibility and necessity DataFrame.

    Args:
        ozone_simulation: The ozone simulation object
        ozone: The ozone variable
        normalize: Boolean indicating whether to normalize the array

    Returns:
        poss_necess_df: DataFrame with possibility and necessity values
    """
    possibility_array = create_possibility_array(ozone_simulation, ozone, normalize=normalize)
    necessity_array = compute_necessity_distr(possibility_array, func="max")
    poss_necess_df = create_dataframe_poss_necess(ozone.terms.keys(), possibility_array, necessity_array)
    return poss_necess_df

def create_dataframe_poss_necess(category_names, possibility_array, necessity_array):
    """
    Create a DataFrame of possibility and necessity values.

    Args:
        category_names: List of category names
        possibility_array: Numpy array of possibility values
        necessity_array: Numpy array of necessity values

    Returns:
        df: DataFrame with possibility and necessity values
    """
    df = pd.DataFrame(index=category_names)
    df['possibility'] = possibility_array
    df['necessity'] = necessity_array
    return df

### 2.8 `plot_ozone_fcst`

Plots the ozone forecast along with possibility and necessity distributions.

In [ ]:
def plot_ozone_fcst(ozone, sim=None, df=None, category_curve_centre=None, plot_necessity=False):
    """
    Plot the ozone forecast with the possibility and necessity of each category.

    Args:
        ozone: The ozone variable
        sim: The ozone simulation object
        df: DataFrame with possibility and necessity values
        category_curve_centre: Dict with category names and their Gaussian curve centers
        plot_necessity: Boolean indicating whether to plot necessity values
    """
    ozone.view(sim=sim)

    if df is not None:
        for i, cat in enumerate(ozone.terms.keys()):
            if plot_necessity:
                label = "Necessity" if i == 0 else ""
                plt.plot(category_curve_centre[cat], df.loc[cat, 'necessity'], marker='_', markersize=22,
                         label=label, zorder=10,
                         markeredgecolor='orchid', markeredgewidth=4)

    # Drawing reference lines
    plt.axvline(x=40, color='k', linestyle=':', linewidth=1.5)
    plt.text(38, 0.85, 'Typical\nBackground', horizontalalignment='right', fontsize=8)
    plt.axvline(x=70, color='magenta', linestyle=':', linewidth=1.5)
    plt.text(72, 0.85, 'NAAQS for Ozone', horizontalalignment='left', fontsize=8)

    plt.xlabel('Ozone Concentration (ppb)')
    plt.ylim(0, 1)
    plt.legend()
    plt.show()

## 3. Defining Input Variables and Universes of Discourse

---

We define the universes of discourse for each input variable based on their expected ranges.

In [ ]:
# Snow depth in mm (0 to 750 mm)
snow_uod = np.arange(0, 750, 5)

# Mean sea-level pressure in Pa (100,000 Pa to 107,000 Pa)
mslp_uod = np.arange(100000, 107000, 500)

# Wind speed in m/s (0 to 20 m/s)
wind_uod = np.arange(0, 20, 0.125)

# Solar radiation in W/m^2 (100 to 1100 W/m^2)
solar_uod = np.arange(100, 1100, 10)

# Define the Antecedents (input variables)
snow = ctrl.Antecedent(snow_uod, 'snow')
mslp = ctrl.Antecedent(mslp_uod, 'mslp')
wind = ctrl.Antecedent(wind_uod, 'wind')
solar = ctrl.Antecedent(solar_uod, 'solar')

# Define the Consequent (output variable)
ozone = ctrl.Consequent(np.arange(20, 140, 1), 'ozone')

## 4. Creating Membership Functions

---

Membership functions define how each point in the input space is mapped to a membership value between 0 and 1.

### 4.1 Snow Membership Functions

In [ ]:
# Define membership functions for snow
snow['negligible'] = fuzz.sigmf(snow.universe, 70, -0.07)
snow['sufficient'] = fuzz.sigmf(snow.universe, 100, 0.07)

# Plotting the snow membership functions
fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
snow.view(ax=ax)
ax.set_xlabel("Snow depth (mm)")
plt.show()

### 4.2 Pressure Membership Functions

In [ ]:
# Define membership functions for mean sea-level pressure (mslp)
mslp['low'] = fuzz.sigmf(mslp.universe, 101300, -0.005)
mslp['average'] = fuzz.gaussmf(mslp.universe, 102900, 800)
mslp['high'] = fuzz.sigmf(mslp.universe, 104500, 0.005)

# Plotting the pressure membership functions
fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
mslp.view(ax=ax)
# Adjust x-axis to display in hPa
ax.set_xticks(np.arange(100000, 107500, 1000))
ax.set_xticklabels(np.arange(1000, 1075, 10))
ax.set_xlabel("Mean Sea-Level Pressure (hPa)")
plt.show()

### 4.3 Wind Speed Membership Functions

In [ ]:
# Define membership functions for wind speed
wind['calm'] = fuzz.sigmf(wind.universe, 2.5, -3.0)
wind['breezy'] = fuzz.sigmf(wind.universe, 2.5, 3.0)

# Plotting the wind speed membership functions
fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
wind.view(ax=ax)
ax.set_xlabel("Wind Speed ($m\,s^{-1}$)")
plt.show()

### 4.4 Solar Radiation Membership Functions

In [ ]:
# Define membership functions for solar radiation
solar['midwinter'] = fuzz.sigmf(solar.universe, 300, -0.03)
solar['winter'] = fuzz.gaussmf(solar.universe, 450, 100)
solar['spring'] = fuzz.gaussmf(solar.universe, 650, 100)
solar['summer'] = fuzz.sigmf(solar.universe, 750, 0.03)

# Plotting the solar radiation membership functions
fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
solar.view(ax=ax)
ax.set_xlabel("Solar Radiation ($W\,m^{-2}$)")
plt.show()

### 4.5 Ozone Membership Functions

In [ ]:
# Define Gaussian parameters for ozone categories
curve_centres = {'background': 40, 'moderate': 52, 'elevated': 67, 'extreme': 95}
sigma_vals = {'background': 6, 'moderate': 5.5, 'elevated': 6, 'extreme': 10}

# Define membership functions for ozone
ozone['background'] = fuzz.gaussmf(ozone.universe, curve_centres['background'], sigma_vals['background'])
ozone['moderate'] = fuzz.gaussmf(ozone.universe, curve_centres['moderate'], sigma_vals['moderate'])
ozone['elevated'] = fuzz.gaussmf(ozone.universe, curve_centres['elevated'], sigma_vals['elevated'])
ozone['extreme'] = fuzz.gaussmf(ozone.universe, curve_centres['extreme'], sigma_vals['extreme'])

# Plotting the ozone membership functions
plot_ozone_fcst(ozone)

## 5. Defining Fuzzy Rules

---

The fuzzy rules define how the input variables are combined to produce the output.

In [ ]:
# Rule 1: If snow is negligible or pressure is low or wind is breezy, then ozone is background
rule1 = ctrl.Rule((snow['negligible'] | mslp['low'] | wind['breezy']), ozone['background'])

# Rule 2: If snow is sufficient and pressure is high and wind is calm and solar is spring, then ozone is extreme
rule2 = ctrl.Rule(snow['sufficient'] & mslp['high'] & wind['calm'] & solar['spring'], ozone['extreme'])

# Rule 3: If snow is sufficient and pressure is high and wind is calm and solar is winter, then ozone is elevated
rule3 = ctrl.Rule(snow['sufficient'] & mslp['high'] & wind['calm'] & solar['winter'], ozone['elevated'])

# Rule 4: If snow is sufficient and pressure is high and wind is calm and solar is midwinter or summer, then ozone is moderate
rule4 = ctrl.Rule(snow['sufficient'] & mslp['high'] & wind['calm'] & (solar['midwinter'] | solar['summer']), ozone['moderate'])

# Rule 5: If snow is sufficient and pressure is average and wind is calm and solar is spring or winter, then ozone is elevated
rule5 = ctrl.Rule(snow['sufficient'] & mslp['average'] & wind['calm'] & (solar['spring'] | solar['winter']), ozone['elevated'])

# Rule 6: If snow is sufficient and pressure is average and wind is calm and solar is midwinter or summer, then ozone is moderate
rule6 = ctrl.Rule(snow['sufficient'] & mslp['average'] & wind['calm'] & (solar['midwinter'] | solar['summer']), ozone['moderate'])

## 6. Creating the Control System

---

We now create the control system and simulation using the defined rules.

In [ ]:
# Create the control system with the defined rules
ozone_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])

# Create a simulation object
ozone_simulation = ctrl.ControlSystemSimulation(ozone_ctrl)

## 7. Running Example Cases

---

We will test the model with some example cases by setting the input variables and computing the output.

### 7.1 High Ozone Case

**Explanation:** In this scenario, all conditions favor high ozone levels—sufficient snow, high pressure, calm wind, and high solar radiation.

In [ ]:
# Example 1: High ozone case
ozone_simulation.input['snow'] = 250        # Snow depth in mm
ozone_simulation.input['mslp'] = 104500     # Pressure in Pa
ozone_simulation.input['wind'] = 1          # Wind speed in m/s
ozone_simulation.input['solar'] = 640       # Solar radiation in W/m^2

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level:", ozone_simulation.output['ozone'])

# Compute possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)

# Plot the ozone forecast
plot_ozone_fcst(ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres)

### 7.2 Background Ozone Case

**Explanation:** In this case, the wind is breezy and the snow depth is low, leading to background ozone levels.

In [ ]:
# Example 2: Background ozone case
ozone_simulation.input['snow'] = 50         # Snow depth in mm
ozone_simulation.input['mslp'] = 102500     # Pressure in Pa
ozone_simulation.input['wind'] = 4          # Wind speed in m/s
ozone_simulation.input['solar'] = 600       # Solar radiation in W/m^2

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level:", ozone_simulation.output['ozone'])

# Compute possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)

# Plot the ozone forecast
plot_ozone_fcst(ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres)

### 7.3 Cusp Case

**Explanation:** This is a borderline case where conditions could lead to moderate or elevated ozone levels.

In [ ]:
# Example 3: Cusp case
ozone_simulation.input['snow'] = 100        # Snow depth in mm
ozone_simulation.input['mslp'] = 104000     # Pressure in Pa
ozone_simulation.input['wind'] = 1.5        # Wind speed in m/s
ozone_simulation.input['solar'] = 500       # Solar radiation in W/m^2

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level:", ozone_simulation.output['ozone'])

# Compute possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)

# Display the possibility and necessity DataFrame
print(poss_necess_df)

# Plot the ozone forecast
plot_ozone_fcst(ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres)

## 8. Evaluating the Model with Data

---

We will now use real data to evaluate the model.

### 8.1 Loading Data

**Note:** The data should contain columns like `snow_depth`, `sea_level_pressure`, `wind_speed`, `solar_radiation`, and `ozone_concentration`.

In [ ]:
# Load the data (assuming 'df_rep.pkl' is available in the specified path)
df_rep = pd.read_pickle(f"../local_data/df_rep_{y}.pkl")
df_rep.head()

### 8.2 Running the Model on Data

In [ ]:
# Initialize columns for forecast and possibility values
ozone_categories = ['background', 'moderate', 'elevated', 'extreme']
columns_to_initialize = ['ozone_forecast'] + [f'possibility_{cat}' for cat in ozone_categories]

# Add new columns to the DataFrame
for col in columns_to_initialize:
    df_rep[col] = np.nan

# Loop through each row and compute the forecast
for i, row in df_rep.iterrows():
    # Set the input values
    ozone_simulation.input['snow'] = row['snow_depth']
    ozone_simulation.input['mslp'] = row['sea_level_pressure']
    ozone_simulation.input['wind'] = row['wind_speed']
    ozone_simulation.input['solar'] = row['solar_radiation']

    # Compute the simulation
    ozone_simulation.compute()

    # Store the forecasted ozone level
    df_rep.at[i, 'ozone_forecast'] = ozone_simulation.output['ozone']

    # Store the possibility values for each category
    for k, v in ozone.terms.items():
        df_rep.at[i, f'possibility_{k}'] = v.membership_value[ozone_simulation]

## 9. Visualizing Results

---

This section will cover the visualization of the model's results.

### 9.1 Time Series Plot of Observed and Forecasted Ozone Levels

In [ ]:
# Create the figure and the first axis
fig, ax1 = plt.subplots(figsize=(12, 6), dpi=300)

# Plot the forecast and observed ozone concentration
ax1.plot(df_rep.index, df_rep['ozone_forecast'], label='Forecast', color='blue')
ax1.plot(df_rep.index, df_rep['ozone_concentration'], label='Observed', color='black')

# Add horizontal lines for NAAQS limit and typical background
ax1.axhline(y=70, color='magenta', linestyle=':', linewidth=1.5, label='NAAQS for Ozone')
ax1.axhline(y=40, color='k', linestyle=':', linewidth=1.5, label='Typical Background')

# Set labels and limits
ax1.set_ylabel('Ozone Concentration (ppb)', fontsize=12)
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylim(20, 100)

# Format the x-axis for dates
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right', fontsize=10)

# Create a second y-axis for the possibility bars
ax2 = ax1.twinx()

# Plot the possibility bars
ax2.bar(df_rep.index, df_rep['possibility_extreme'], color='red', alpha=0.6, label='Possibility of Extreme Ozone')
ax2.bar(df_rep.index, df_rep['possibility_elevated'], color='green', alpha=0.4, label='Possibility of Elevated Ozone')
ax2.bar(df_rep.index, df_rep['possibility_moderate'], color='orange', alpha=0.3, label='Possibility of Moderate Ozone')
ax2.bar(df_rep.index, df_rep['possibility_background'], color='blue', alpha=0.2, label='Possibility of Background Ozone')

# Set the second y-axis label and limits
ax2.set_ylabel('Possibility', fontsize=12)
ax2.set_ylim(0, 1)

# Combine legends from both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=10)

plt.title('Observed and Forecasted Ozone Levels with Possibility Distributions')
plt.tight_layout()
plt.show()

### 9.2 Detailed Analysis for Specific Dates

Let's analyze the possibility and necessity distributions for specific dates.

In [ ]:
# Define the date to analyze
month = 'Feb'
day = 27
year = maybe_amend_year(month, y)

# Generate the range dictionary and strings
range_dict = range_from_gaussian_params(curve_centres, sigma_vals)
range_str_dict = generate_str_from_range(range_dict)

# Extract the percentiles for the date
percentiles = df_rep.rank(pct=True)
pc_dict = {k: percentiles.loc[f'{year}-{month}-{day}'][k] for k in percentiles.columns if 'possibility_' in k}

# Plot the possibility and necessity distributions
plot_possibility_necessity(df_rep, ozone, month=month, day=day, range_dict=range_dict,
                           range_str_dict=range_str_dict, percentiles=pc_dict)

## 10. Custom Cases Using Fake Data

---

Below, we'll create several custom cases with different input conditions that weren't covered in the original examples. We'll add these at the end of your notebook to showcase how the model responds to various scenarios.

### Custom Case 1: Moderate Ozone with High Winds

**Conditions:**
- Snow depth: 200 mm (sufficient snow)
- Mean sea-level pressure: 103500 Pa (average pressure)
- Wind speed: 6 m/s (breezy wind)
- Solar radiation: 550 W/m² (winter solar radiation)

In [2]:
# Custom Case 1: Moderate Ozone with High Winds
ozone_simulation.input['snow'] = 200        # Snow depth in mm (sufficient)
ozone_simulation.input['mslp'] = 103500     # Pressure in Pa (average)
ozone_simulation.input['wind'] = 6          # Wind speed in m/s (breezy)
ozone_simulation.input['solar'] = 550       # Solar radiation in W/m² (winter)

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level (Custom Case 1):", ozone_simulation.output['ozone'])

# Plot the ozone forecast with possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)
plot_ozone_fcst(
    ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres
)

NameError: name 'ozone_simulation' is not defined

**Explanation:**

- Snow depth is sufficient, which can contribute to higher ozone levels.
- Mean sea-level pressure is average, not strongly favoring high ozone.
- Wind speed is breezy, which tends to disperse pollutants and reduce ozone accumulation.
- Solar radiation is moderate for winter.

We might expect the ozone level to be in the moderate category due to the opposing effects of sufficient snow and breezy winds.

### Custom Case 2: Low Ozone in Midwinter with Calm Winds

**Conditions:**
- Snow depth: 0 mm (negligible snow)
- Mean sea-level pressure: 102000 Pa (low pressure)
- Wind speed: 2 m/s (calm wind)
- Solar radiation: 250 W/m² (midwinter solar radiation)

In [ ]:
# Custom Case 2: Low Ozone in Midwinter with Calm Winds
ozone_simulation.input['snow'] = 0          # Snow depth in mm (negligible)
ozone_simulation.input['mslp'] = 102000     # Pressure in Pa (low)
ozone_simulation.input['wind'] = 2          # Wind speed in m/s (calm)
ozone_simulation.input['solar'] = 250       # Solar radiation in W/m² (midwinter)

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level (Custom Case 2):", ozone_simulation.output['ozone'])

# Plot the ozone forecast with possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)
plot_ozone_fcst(
    ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres
)

**Explanation:**

- Snow depth is negligible, reducing the potential for high ozone levels.
- Mean sea-level pressure is low, associated with more dynamic atmospheric conditions.
- Wind speed is calm, which could allow accumulation, but the lack of snow diminishes this effect.
- Solar radiation is low due to midwinter conditions.

We expect the ozone level to be in the background category.


### Custom Case 3: Elevated Ozone with Mixed Conditions

**Conditions:**
- Snow depth: 150 mm (sufficient snow)
- Mean sea-level pressure: 105000 Pa (high pressure)
- Wind speed: 3 m/s (calm wind)
- Solar radiation: 600 W/m² (approaching spring)

In [ ]:
# Custom Case 3: Elevated Ozone with Mixed Conditions
ozone_simulation.input['snow'] = 150        # Snow depth in mm (sufficient)
ozone_simulation.input['mslp'] = 105000     # Pressure in Pa (high)
ozone_simulation.input['wind'] = 3          # Wind speed in m/s (calm)
ozone_simulation.input['solar'] = 600       # Solar radiation in W/m² (approaching spring)

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level (Custom Case 3):", ozone_simulation.output['ozone'])

# Plot the ozone forecast with possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)
plot_ozone_fcst(
    ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres
)

**Explanation:**

- Snow depth is sufficient.
- Mean sea-level pressure is very high, favoring ozone formation.
- Wind speed is calm.
- Solar radiation is increasing as we approach spring, enhancing photochemical reactions.

We might expect the ozone level to be in the elevated or possibly extreme category, depending on the model's interpretation.

### Custom Case 4: Background Ozone with High Solar Radiation

**Conditions:**
- Snow depth: 50 mm (negligible snow)
- Mean sea-level pressure: 103000 Pa (average pressure)
- Wind speed: 4 m/s (breezy wind)
- Solar radiation: 800 W/m² (summer-like solar radiation)


In [ ]:
# Custom Case 4: Background Ozone with High Solar Radiation
ozone_simulation.input['snow'] = 50         # Snow depth in mm (negligible)
ozone_simulation.input['mslp'] = 103000     # Pressure in Pa (average)
ozone_simulation.input['wind'] = 4          # Wind speed in m/s (breezy)
ozone_simulation.input['solar'] = 800       # Solar radiation in W/m² (summer-like)

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level (Custom Case 4):", ozone_simulation.output['ozone'])

# Plot the ozone forecast with possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)
plot_ozone_fcst(
    ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres
)

**Explanation:**

- Snow depth is negligible.
- Mean sea-level pressure is average.
- Wind speed is breezy.
- Solar radiation is high, typical of summer.

Despite the high solar radiation, the lack of snow and breezy winds suggest that ozone levels should remain in the background category according to the model rules.

### Custom Case 5: Extreme Ozone with Midwinter Solar Radiation

**Conditions:**
- Snow depth: 500 mm (very sufficient snow)
- Mean sea-level pressure: 104800 Pa (high pressure)
- Wind speed: 0.8 m/s (very calm wind)
- Solar radiation: 350 W/m² (midwinter solar radiation)

In [ ]:
# Custom Case 5: Extreme Ozone with Midwinter Solar Radiation
ozone_simulation.input['snow'] = 500        # Snow depth in mm (very sufficient)
ozone_simulation.input['mslp'] = 104800     # Pressure in Pa (high)
ozone_simulation.input['wind'] = 0.8        # Wind speed in m/s (very calm)
ozone_simulation.input['solar'] = 350       # Solar radiation in W/m² (midwinter)

# Compute the simulation
ozone_simulation.compute()

# Output the predicted ozone level
print("Predicted Ozone Level (Custom Case 5):", ozone_simulation.output['ozone'])

# Plot the ozone forecast with possibility and necessity
poss_necess_df = compute_possibility_necessity_df(ozone_simulation, ozone)
plot_ozone_fcst(
    ozone, sim=ozone_simulation, df=poss_necess_df, category_curve_centre=curve_centres
)

**Explanation:**

- Snow depth is very high, significantly contributing to ozone formation.
- Mean sea-level pressure is high, indicating stable conditions.
- Wind speed is very calm, allowing pollutants to accumulate.
- Solar radiation is low due to midwinter conditions.

Despite low solar radiation, the other conditions are strongly favorable for ozone formation. According to the model's rules, this could result in an elevated or even extreme ozone level.